In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers, models
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report

In [2]:
mnist=tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test)=mnist.load_data()

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [3]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

In [4]:
x_train, x_test=x_train/255.0, x_test/255.0

In [5]:
x_train=x_train.reshape((x_train.shape[0], 28, 28, 1))
x_test=x_test.reshape((x_test.shape[0], 28, 28, 1))

In [6]:
#CNN modeli oluştur

model=models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1)),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history=model.fit(x_train, y_train, epochs=5, validation_data=(x_test, y_test))

Epoch 1/5
1202/1875 ━━━━━━━━━━━━━━━━━━━━ 20s 30ms/step - accuracy: 0.8573 - loss: 0.4500

In [ ]:
y_pred=model.predict(x_test)
y_pred_classes=np.argmax(y_pred, axis=1)

conf_matrix=confusion_matrix(y_test, y_pred_classes)
print(conf_matrix)

plt.figure(figsize=(10,8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title("Confusion Matrix")
plt.show()

print(classification_report(y_test, y_pred_classes))

In [ ]:
#modeli kaydet
model.save('mnist_model.h5')

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import cv2
import numpy as np
model=load_model('mnist_model.h5')

In [ ]:
def load_and_prepare_image(img_path):
    img=cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
      raise ValueError("Görüntü yüklenemedi")

    img=cv2.resize(img, (28,28))  #çünkü model 28*28 lik ayarlandı
    img=img/255.0
    img=img.reshape((1,28,28,1))
    return img

In [ ]:
def classify_image(img_path):
    img=load_and_prepare_image(img_path)
    prediction=model.predict(img)
    predicted_class=np.argmax(prediction)
    return predicted_class

In [ ]:
img_path="/content/yedi.png"
predicted_class=classify_image(img_path)
print(f"Tahmin edilen sınıf: {predicted_class}")